In [1]:
import torch
import torchvision
import torch.nn.functional as F
import torch.nn as nn
import torchtoolbox.transform as transforms
from torch.utils.data import Dataset, DataLoader, Subset
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold
import pandas as pd
import numpy as np
import gc
import os
import cv2
import time
import datetime
import warnings
import random
import matplotlib.pyplot as plt
import seaborn as sns
from efficientnet_pytorch import EfficientNet
%matplotlib inline

In [2]:
warnings.simplefilter('ignore')
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(47)


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
class MelanomaDataset(Dataset):
    def __init__(self, df: pd.DataFrame, imfolder: str, train: bool = True, transforms = None, meta_features = None):
        """
        Class initialization
        Args:
            df (pd.DataFrame): DataFrame with data description
            imfolder (str): folder with images
            train (bool): flag of whether a training dataset is being initialized or testing one
            transforms: image transformation method to be applied
            meta_features (list): list of features with meta information, such as sex and age
            
        """
        self.df = df
        self.imfolder = imfolder
        self.transforms = transforms
        self.train = train
        self.meta_features = meta_features
        
    def __getitem__(self, index):
        im_path = os.path.join(self.imfolder, self.df.iloc[index]['image_name'] + '.jpg')
        x = cv2.imread(im_path)
        meta = np.array(self.df.iloc[index][self.meta_features].values, dtype=np.float32)

        if self.transforms:
            x = self.transforms(x)
            
        if self.train:
            y = self.df.iloc[index]['target']
            return (x, meta), y
        else:
            return (x, meta)
    
    def __len__(self):
        return len(self.df)

In [5]:
class Net(nn.Module):
    def __init__(self, arch, n_meta_features: int):
        super(Net, self).__init__()
        self.arch = arch
        if 'ResNet' in str(arch.__class__):
            self.arch.fc = nn.Linear(in_features=512, out_features=500, bias=True)
        if 'EfficientNet' in str(arch.__class__):
            self.arch._fc = nn.Linear(in_features=1280, out_features=500, bias=True)
        self.meta = nn.Sequential(nn.Linear(n_meta_features, 500),
                                  nn.BatchNorm1d(500),
                                  nn.ReLU(),
                                  nn.Dropout(p=0.2),
                                  nn.Linear(500, 250),  # FC layer output will have 250 features
                                  nn.BatchNorm1d(250),
                                  nn.ReLU(),
                                  nn.Dropout(p=0.2))
        self.ouput1 = nn.Linear(500 + 250, 256)
        self.ouput2 = nn.Linear(256,1)
        
    def forward(self, inputs):
        """
        No sigmoid in forward because we are going to use BCEWithLogitsLoss
        Which applies sigmoid for us when calculating a loss
        """
        x, meta = inputs
        cnn_features = self.arch(x)
        meta_features = self.meta(meta)
        features = torch.cat((cnn_features, meta_features), dim=1)
        output = self.ouput2(self.ouput1(features))
        return output

In [6]:
train_transform = transforms.Compose([
    
    transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),

    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
])
test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
])

In [7]:
arch = EfficientNet.from_pretrained('efficientnet-b1')

Loaded pretrained weights for efficientnet-b1


In [8]:
BASEPATH = "/home/abhijit/Documents/skin_cancer_2/jpeg/"
train_df = pd.read_csv(os.path.join(BASEPATH, 'train.csv'))
test_df  = pd.read_csv(os.path.join(BASEPATH, 'test.csv'))

In [9]:
train_df.head()

,image_name,patient_id,sex,age_approx,anatom_site_general_challenge,diagnosis,benign_malignant,target
0,ISIC_2637011,IP_7279968,male,45.0,head/neck,unknown,benign,0
1,ISIC_0015719,IP_3075186,female,45.0,upper extremity,unknown,benign,0
2,ISIC_0052212,IP_2842074,female,50.0,lower extremity,nevus,benign,0
3,ISIC_0068279,IP_6890425,female,45.0,head/neck,unknown,benign,0
4,ISIC_0074268,IP_8723313,female,55.0,upper extremity,unknown,benign,0


In [10]:
test_df.head()

,image_name,patient_id,sex,age_approx,anatom_site_general_challenge
0,ISIC_0052060,IP_3579794,male,70.0,NaN
1,ISIC_0052349,IP_7782715,male,40.0,lower extremity
2,ISIC_0058510,IP_7960270,female,55.0,torso
3,ISIC_0073313,IP_6375035,female,50.0,torso
4,ISIC_0073502,IP_0589375,female,45.0,lower extremity


In [11]:
from sklearn.model_selection import train_test_split
#train_df, valid_df = train_test_split(train_df, test_size=0.1, random_state= 42)

In [12]:
train_df.head()

,image_name,patient_id,sex,age_approx,anatom_site_general_challenge,diagnosis,benign_malignant,target
0,ISIC_2637011,IP_7279968,male,45.0,head/neck,unknown,benign,0
1,ISIC_0015719,IP_3075186,female,45.0,upper extremity,unknown,benign,0
2,ISIC_0052212,IP_2842074,female,50.0,lower extremity,nevus,benign,0
3,ISIC_0068279,IP_6890425,female,45.0,head/neck,unknown,benign,0
4,ISIC_0074268,IP_8723313,female,55.0,upper extremity,unknown,benign,0


In [13]:
concat = pd.concat([train_df['anatom_site_general_challenge'], test_df['anatom_site_general_challenge']], ignore_index=True)
dummies = pd.get_dummies(concat, dummy_na=True, dtype=np.uint8, prefix='site')
train_df = pd.concat([train_df, dummies.iloc[:train_df.shape[0]]], axis=1)
#valid_df = pd.concat([valid_df, dummies.iloc[train_df.shape[0]:train_df.shape[0] + valid_df.shape[0]]], axis=1)
test_df = pd.concat([test_df, dummies.iloc[train_df.shape[0]:].reset_index(drop=True)], axis=1)

# Sex features
train_df['sex'] = train_df['sex'].map({'male': 1, 'female': 0})
#valid_df['sex'] = valid_df['sex'].map({'male': 1, 'female': 0})
test_df['sex'] = test_df['sex'].map({'male': 1, 'female': 0})
train_df['sex'] = train_df['sex'].fillna(-1)
#valid_df['sex'] = valid_df['sex'].fillna(-1)
test_df['sex'] = test_df['sex'].fillna(-1)

# Age features
train_df['age_approx'] /= train_df['age_approx'].max()
#valid_df['age_approx'] /= train_df['age_approx'].max()
test_df['age_approx'] /= test_df['age_approx'].max()
train_df['age_approx'] = train_df['age_approx'].fillna(0)
test_df['age_approx'] = test_df['age_approx'].fillna(0)
#valid_df['age_approx'] = valid_df['age_approx'].fillna(0)

train_df['patient_id'] = train_df['patient_id'].fillna(0)
#valid_df['patient_id'] = valid_df['patient_id'].fillna(0)


In [14]:
train_df.head()

,image_name,patient_id,sex,age_approx,anatom_site_general_challenge,diagnosis,benign_malignant,target,site_head/neck,site_lower extremity,site_oral/genital,site_palms/soles,site_torso,site_upper extremity,site_nan
0,ISIC_2637011,IP_7279968,1.0,0.500000,head/neck,unknown,benign,0,1,0,0,0,0,0,0
1,ISIC_0015719,IP_3075186,0.0,0.500000,upper extremity,unknown,benign,0,0,0,0,0,0,1,0
2,ISIC_0052212,IP_2842074,0.0,0.555556,lower extremity,nevus,benign,0,0,1,0,0,0,0,0
3,ISIC_0068279,IP_6890425,0.0,0.500000,head/neck,unknown,benign,0,1,0,0,0,0,0,0
4,ISIC_0074268,IP_8723313,0.0,0.611111,upper extremity,unknown,benign,0,0,0,0,0,0,1,0


In [15]:
train_df, valid_df = train_test_split(train_df, test_size=0.1, random_state= 42)

In [16]:
train_df.head()

,image_name,patient_id,sex,age_approx,anatom_site_general_challenge,diagnosis,benign_malignant,target,site_head/neck,site_lower extremity,site_oral/genital,site_palms/soles,site_torso,site_upper extremity,site_nan
4237,ISIC_1376797,IP_5331792,1.0,0.611111,lower extremity,unknown,benign,0,0,1,0,0,0,0,0
7341,ISIC_2295004,IP_8702124,0.0,0.555556,torso,unknown,benign,0,0,0,0,0,1,0,0
6916,ISIC_2162458,IP_7700121,1.0,0.444444,torso,unknown,benign,0,0,0,0,0,1,0,0
14778,ISIC_4525317,IP_2738587,1.0,0.777778,torso,melanoma,malignant,1,0,0,0,0,1,0,0
15757,ISIC_4819371,IP_7530983,0.0,0.555556,upper extremity,unknown,benign,0,0,0,0,0,0,1,0


In [17]:
valid_df.head()

,image_name,patient_id,sex,age_approx,anatom_site_general_challenge,diagnosis,benign_malignant,target,site_head/neck,site_lower extremity,site_oral/genital,site_palms/soles,site_torso,site_upper extremity,site_nan
8231,ISIC_2566319,IP_9258690,1.0,0.388889,upper extremity,nevus,benign,0,0,0,0,0,0,1,0
21862,ISIC_6617618,IP_5350484,1.0,0.333333,lower extremity,unknown,benign,0,0,1,0,0,0,0,0
3058,ISIC_1018102,IP_3723085,0.0,0.444444,torso,unknown,benign,0,0,0,0,0,1,0,0
7474,ISIC_2334999,IP_7779275,0.0,0.500000,upper extremity,unknown,benign,0,0,0,0,0,0,1,0
3470,ISIC_1151651,IP_9754730,0.0,0.611111,head/neck,nevus,benign,0,1,0,0,0,0,0,0


In [18]:
test_df.head()

,image_name,patient_id,sex,age_approx,anatom_site_general_challenge,site_head/neck,site_lower extremity,site_oral/genital,site_palms/soles,site_torso,site_upper extremity,site_nan
0,ISIC_0052060,IP_3579794,1,0.777778,NaN,0,0,0,0,0,0,1
1,ISIC_0052349,IP_7782715,1,0.444444,lower extremity,0,1,0,0,0,0,0
2,ISIC_0058510,IP_7960270,0,0.611111,torso,0,0,0,0,1,0,0
3,ISIC_0073313,IP_6375035,0,0.555556,torso,0,0,0,0,1,0,0
4,ISIC_0073502,IP_0589375,0,0.500000,lower extremity,0,1,0,0,0,0,0


In [19]:
meta_features = ['sex', 'age_approx'] + [col for col in train_df.columns if 'site_' in col]
meta_features.remove('anatom_site_general_challenge')

In [20]:
test = MelanomaDataset(df=test_df,
                       imfolder= "/home/abhijit/Documents/skin_cancer_2/jpeg/test/", 
                       train=False,
                       transforms=train_transform,  # For TTA
                       meta_features=meta_features)

In [21]:

from torch.optim import lr_scheduler
arch = EfficientNet.from_pretrained('efficientnet-b1')
model = Net(arch=arch, n_meta_features=len(meta_features))  # New model for each fold
model = model.to(device)
    
optim = torch.optim.Adam(model.parameters(), lr=0.00025)
scheduler = lr_scheduler.StepLR(optim, 6, gamma=0.5, last_epoch=-1)
criterion = nn.BCEWithLogitsLoss()

Loaded pretrained weights for efficientnet-b1


In [22]:
model

Net(
  (arch): EfficientNet(
    (_conv_stem): Conv2dStaticSamePadding(
      3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
      (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
    )
    (_bn0): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
    (_blocks): ModuleList(
      (0): MBConvBlock(
        (_depthwise_conv): Conv2dStaticSamePadding(
          32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
          (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
        )
        (_bn1): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
        (_se_reduce): Conv2dStaticSamePadding(
          32, 8, kernel_size=(1, 1), stride=(1, 1)
          (static_padding): Identity()
        )
        (_se_expand): Conv2dStaticSamePadding(
          8, 32, kernel_size=(1, 1), stride=(1, 1)
          (static_padding): Identity()
        )
        (_project_con

In [23]:
train = MelanomaDataset(df=train_df, 
                            imfolder="/home/abhijit/Documents/skin_cancer_2/jpeg/train/", 
                            train=True, 
                            transforms=train_transform,
                            meta_features=meta_features)
val = MelanomaDataset(df=valid_df, 
                            imfolder="/home/abhijit/Documents/skin_cancer_2/jpeg/train/", 
                            train=True, 
                            transforms=train_transform,
                            meta_features=meta_features)

In [24]:
train_loader = DataLoader(dataset=train, batch_size=16, shuffle=True, num_workers=1)
val_loader = DataLoader(dataset=val, batch_size=16, shuffle=False, num_workers=1)
test_loader = DataLoader(dataset=test, batch_size=1, shuffle=False, num_workers=1)

In [25]:
for i, data in enumerate(val_loader):
    X, Y = data
    [image, meta] = X
    
    print(image.shape)
    print(meta.shape)
    
    if i == 2:
        break

torch.Size([16, 3, 256, 256])
torch.Size([16, 9])
torch.Size([16, 3, 256, 256])
torch.Size([16, 9])
torch.Size([16, 3, 256, 256])
torch.Size([16, 9])


In [45]:
epochs = 12
best_acc = 0
best_acc_val = 0

for epoch in range(epochs):
    start_time = time.time()
    correct = 0
    epoch_loss = 0
    model.train()
    m = 0   
    for x, y in train_loader:
        x[0] = torch.tensor(x[0], device=device, dtype=torch.float32)
        x[1] = torch.tensor(x[1], device=device, dtype=torch.float32)
        y = torch.tensor(y, device=device, dtype=torch.float32)
        optim.zero_grad()
        z = model(x)
        loss = criterion(z, y.unsqueeze(1))
        loss.backward()
        optim.step()
        pred = torch.round(torch.sigmoid(z))  # round off sigmoid to obtain predictions
        correct += (pred.cpu() == y.cpu().unsqueeze(1)).sum().item()  # tracking number of correctly predicted samples
        epoch_loss += loss.item()
        
        m += 1
        #if m ==1:
         #   break
        
    train_acc = correct / (29813)
    
    
    
    val_len = 3312
                           
    correct = 0   
    m = 0
    with torch.no_grad():
        # Predicting on validation set once again to obtain data for OOF
        for j, (x_val, y_val) in enumerate(val_loader):
            x_val[0] = torch.tensor(x_val[0], device=device, dtype=torch.float32)
            x_val[1] = torch.tensor(x_val[1], device=device, dtype=torch.float32)
            y_val = torch.tensor(y_val, device=device, dtype=torch.float32)
            z_val = model(x_val)
            pred = torch.round(torch.sigmoid(z_val))
            correct += (pred.cpu() == y_val.cpu().unsqueeze(1)).sum().item()
            m += 1
            if m== 180:
                break
    valid_acc = correct / 2880
                           
    if train_acc > best_acc:
        best_acc = train_acc
        if valid_acc > best_acc_val:
            best_acc_val = valid_acc
            torch.save(model.state_dict(), '/home/abhijit/Documents/skin_cancer_2/jpeg/efficienNet')
    print("Epoch loss : %f Train Acc: %f Valid Acc: %f\n" %(epoch_loss, train_acc, valid_acc))
    
    scheduler.step()
        

Epoch loss : 156.220832 Train Acc: 0.982524 Valid Acc: 0.980903

Epoch loss : 144.512628 Train Acc: 0.982558 Valid Acc: 0.980903

Epoch loss : 137.490796 Train Acc: 0.982457 Valid Acc: 0.980903

Epoch loss : 133.250955 Train Acc: 0.982558 Valid Acc: 0.980903

Epoch loss : 128.034155 Train Acc: 0.982759 Valid Acc: 0.981250

Epoch loss : 124.337725 Train Acc: 0.982591 Valid Acc: 0.980208

Epoch loss : 111.036931 Train Acc: 0.982625 Valid Acc: 0.981944

Epoch loss : 105.395813 Train Acc: 0.982960 Valid Acc: 0.980208

Epoch loss : 96.976573 Train Acc: 0.983464 Valid Acc: 0.979861

Epoch loss : 89.164791 Train Acc: 0.984604 Valid Acc: 0.978472

Epoch loss : 83.629818 Train Acc: 0.985208 Valid Acc: 0.982292

Epoch loss : 79.512323 Train Acc: 0.985610 Valid Acc: 0.982986



In [25]:
model.load_state_dict(torch.load('/home/abhijit/Documents/skin_cancer_2/jpeg/efficienNet'))

<All keys matched successfully>

In [28]:
tta_preds = torch.zeros((len(test), 1), dtype=torch.float32, device=device)
with torch.no_grad():
    model.eval()
    for i, x_test in enumerate(test_loader):
        x_test[0] = torch.tensor(x_test[0], device=device, dtype=torch.float32)
        x_test[1] = torch.tensor(x_test[1], device=device, dtype=torch.float32)
        z_test = model(x_test)
        z_test = torch.sigmoid(z_test)
        tta_preds[i*test_loader.batch_size:i*test_loader.batch_size + x_test[0].shape[0]] += z_test
        
        if (i % 100) == 0:
            print(i)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900


In [31]:
sub = pd.read_csv('/home/abhijit/Documents/skin_cancer_2/jpeg/output.csv')
sub['target'] = tta_preds.cpu().numpy().reshape(-1,)
sub.to_csv('/home/abhijit/Documents/skin_cancer_2/jpeg/submission.csv', index=False)